In [1]:
import numpy as np # no use until now
import random # used for random sampling of values where applicable
import math # no use untill now
import pandas as pd # to read the excel file 
import os # to carry os operations

In [2]:
def make_templates(file_name="templates_for_dialogue_self_play.xlsx",sheet_name="MAKE_TRANSACTION",previous_dictionary=None) :
    if previous_dictionary == None :
        template_dictionary = dict()
    else :
        template_dictionary = previous_dictionary
    
    df = pd.read_excel(file_name,sheet_name)
    for index,row in df.iterrows() :
        if not pd.isnull(row["LABEL"]) :
            if row["LABEL"] not in template_dictionary.keys() :
                template_sentences = list()
            else :
                template_sentences = template_dictionary[row["LABEL"]]
                
            if not pd.isnull(row["TEXT-EN (Marco)"]) :
                template_sentences.append(row["TEXT-EN (Marco)"])
            if not pd.isnull(row["TEXT-EN (Sourabh)"]) :
                template_sentences.append(row["TEXT-EN (Sourabh)"])
                
            template_dictionary[row["LABEL"]] = template_sentences
    
    return template_dictionary

In [3]:
transaction_templates = make_templates(file_name="templates_for_dialogue_self_play.xlsx",
                                     sheet_name="MAKE_TRANSACTION",
                                     previous_dictionary=None)

In [4]:
class Action(object) :
    def __init__(self,actor=None,action=None,slots=None,values=None,message=None,description=None,templates=None) :
        
        self.actor = actor # who performed the action
        self.action = action # what action was performed
        self.slots = slots # what slot was dealt with 
        self.values = values # what was the value with this slot
        self.message = message # Any particular message related to the action
        self.description = description # This contain the description of the action and is never intended to be shown or appear in the actual conversation
        
        dictionary_key = self.action
        
        if self.slots :
            if len(self.slots) > 0 :
                for slot in self.slots :
                    
                    if slot == "intent" :
                        
                        if self.actor == "User" :
                            
                            dictionary_key += "-" + slot + "_" + self.values["intent"]
                            
                        else :
                            
                            dictionary_key += "-" + slot
                    else :
                        
                        dictionary_key += "-" + slot
        
        
        # if the dictionary_key exists in the template dictionary then get a template other wise set template = the action message
        if dictionary_key in templates.keys() :
            self.template = random.sample(templates[dictionary_key],1)[0]
        else :
            self.template = self.message
        
    # standard actions to perform
    def get_actor(self) :
        return self.actor
    
    def get_action(self) :
        return self.action
    
    def get_slots(self) :
        return self.slots
    
    def get_values(self) :
        return self.values
    
    def get_message(self) :
        return self.message
    
    def get_description(self) :
        return self.description
    
    # when called , construct a dialog from the slots and give it to the user
    def get_dialog(self,with_actor=True) :

            
        # first split the template into a list of words
        words = self.template.strip().split()
        sentence = None
        
        # only go to this loop if the actor is a User or Bot , in case of API go to else statement which will just append action and message
        if self.actor == "User" or self.actor == "Bot" :
            if self.values :
                for slot,value in self.values.items() :
                    search_slot = "{" + slot + "}"
                    if search_slot in words :
                        slot_index = words.index(search_slot)
                        words.insert(slot_index,str(value))
                
                        words.pop(slot_index+1)
                    sentence = " ".join(words)
                
            # if it's a request action then get a template requesting the slots
            elif self.action == "request" :
                
                sentence = self.template
            
            # if it's a end_call then show the action and the message given
            elif self.action == "end_call" :
                
                sentence = self.action + " " + self.message
            
            else :
                
                sentence = self.message
        
        else:
            
            sentence = self.action + " " + self.message
        if with_actor :
            return self.actor + " : " + sentence
        else :
            return sentence

In [5]:
action = Action(actor="Bot",action="api_call",slots=["user_account","destination_name","amount"],
                values={"user_account" : "savings","destination_name" : "sourabh","amount" : "1000"},
                message="api_name:transaction, user_account:{}, destination_name:{}, amount:{}".format("savings","sourabh","1000"),
                description="API",templates=transaction_templates)
print(action.get_dialog())

Bot : api_name:transaction, user_account:savings, destination_name:sourabh, amount:1000


In [6]:
class Transaction_user() :
    def __init__(self,templates=None) :
        
        # Below is the available pool of values from which we will create a Custom user for the transaction
        self.user_names = ["Sourabh","Serra","Simone","Marco","Vevake","Matteo","Tahir","Samuel"]
        self.user_accounts = ["Savings","Credit","Checkin"]
        self.transaction_limit = [1000,2000,5000]
        self.user_balances = [400,1300,3000,8000]
        self.transfer_amt = [200,800,1200,1600,2400,4500,9000]
        self.slots = ["user_account","destination_name","amount"]
        self.templates = templates
        self.priority_states = list()
        self.priority_actions = dict()
        
        # create the custom user
        self.user = dict()
        self.create_user_profile()
    
    def sort_my_slots(self,slots_given) :
        
        
        if slots_given :
            
            slots_sorted = list()
            
            if "user_account" in slots_given :
                slots_sorted.append("user_account")
                slots_given.remove("user_account")
            
            if "destination_name" in slots_given :
                slots_sorted.append("destination_name")
                slots_given.remove("destination_name")
            
            if "amount" in slots_given :
                slots_sorted.append("amount")
                slots_given.remove("amount")
        
            for slot in slots_given :
                slots_sorted.append(slot)
        else :
            slots_sorted = list()
        
        return slots_sorted
    
    def create_user_profile(self) :
        
        # Every value is assigned randomly 
        
        # selectinng name of sender and reciever
        
        names = random.sample(self.user_names,2)
        
        self.user["name"] = names[0]
        
        self.user["destination_name"] = names[1]
        number_of_destination_names = random.randint(1,len(self.user_names))
        self.user["destination_names"] = random.sample(self.user_names,number_of_destination_names)
        
        #selecting the usr_account to make the transaction from
        self.user["user_account"] = random.sample(self.user_accounts,1)[0]
        
        number_of_user_accounts = random.randint(1,len(self.user_accounts))
        self.user["user_accounts"] = random.sample(self.user_accounts,number_of_user_accounts)
        
        # selecting the amount to be transfered
        self.user["amount"] = random.sample(self.transfer_amt,1)[0]
        
        # selecting the balance of the user
        self.user["balance"] = random.sample(self.user_balances,1)[0]
        
        # selecting the limit of the user
        self.user["limit"] = random.sample(self.transaction_limit,1)[0]
        
        # setting up the max_transferable amount
        self.user["max_transferable_amt"] = min(self.user["limit"],self.user["balance"])
        
        # setting up the intent
        self.user["intent"] = "transaction"
    
    # Returns the respective value of the slot
    def get_value(self,slot_asked) :
        
        return self.user[slot_asked]
    
    def perform_random_action(self,bot_action) :
        
        values_to_give = dict()
        actual_actor = None
        actual_action = None
        
        if bot_action.get_description() == "API_CALL" :
            
            actual_actor = "API"
            actual_action = "api_response"
            accept_message = "api_result:success"
            reject_message = "api_result:failed"
        
        elif bot_action.get_description() == "CHANGE_ACCOUNT" :
            
            actual_actor = "User"
            actual_action = "api_response"
            accept_message = "accept"
            reject_message = "reject"
            
            new_account = random.sample(self.user_accounts,1)[0]
            while new_account == self.user["user_account"] :
                new_account = random.sample(self.user_accounts,1)[0]
                
            self.user["user_account"] = new_account
            values_to_give["user_account"] = new_account
        
        elif bot_action.get_description() == "CHANGE_AMOUNT" :
            
            actual_actor = "User"
            actual_response = "api_response"
            accept_message = "accept"
            reject_message = "reject"
            self.user["amount"] = self.user["max_transferable_amt"]
            values_to_give["amount"] = self.user["max_transferable_amt"]
        
        elif bot_action.get_description() == "CHANGE_DESTINATION_NAME" :
            
            actual_actor = "User"
            accept_message = "accept"
            reject_message = "reject"
            
            new_destination_name = random.sample(self.user_names,1)[0]
            
            while new_destination_name == self.user["name"] or new_destination_name == self.user["destination_name"] :
                new_destination_name = random.sample(self.user_names,1)[0]
                
            self.user["destination_name"] = new_destination_name
            values_to_give["destination_name"] = new_destination_name
        
        else :
            actual_actor = "User"
            accept_message = "accept"
            reject_message = "reject"
            
        toss = random.randint(0,1)
        if toss == 1 :
            user_action = Action(actor=actual_actor,
                                 action="inform",
                                 slots=None,
                                 values=values_to_give,
                                 message=accept_message,
                                 templates=self.templates)
        else :
            user_action = Action(actor=actual_actor,
                                 action=actual_action,
                                 slots=None,
                                 values=values_to_give,
                                 message=reject_message,
                                 templates=self.templates)
        return user_action
    # This is the function that converses with the bot through 'Action' Objects
    def speak(self,bot_action) :
        user_action = None
        if bot_action.get_action() == "api_call" :
            
            user_action = self.api_response(bot_action)            

        elif bot_action.get_action() == "request" :
            
            if bot_action.get_slots() != None :
                
                if bot_action.get_slots()[0] != "intent" :
                    
                    user_value = self.get_value(bot_action.get_slots()[0])
                    user_action = Action(actor="User",
                                         action="inform",
                                         slots=bot_action.get_slots(),
                                         values={bot_action.get_slots()[0] : user_value},
                                         message="Providing value for {}".format(bot_action.get_slots()[0]),
                                         templates=self.templates)
                
                else :
                    
                    number_of_slots = random.randint(0,len(self.slots))
                    slots_to_inform = random.sample(self.slots,number_of_slots)
                    all_slots = ["intent"] + self.sort_my_slots(slots_to_inform)
                    
                    values_to_inform = dict()
                    
                    for slot in all_slots :
                        values_to_inform[slot] = self.user[slot]
                    
                    user_action = Action(actor="User",
                                         action="inform",
                                         slots=all_slots,
                                         values=values_to_inform,
                                         message="Providing intent",
                                         templates=self.templates)
            else :
                user_action = self.perform_random_action(bot_action)
        
        elif bot_action.get_action() == "api_call" :
            
            user_action = self.perform_random_action(bot_action) 
        
        else :
            
            user_action = Action(actor="User",
                                 action=None,
                                 slots=None,
                                 values=None,
                                 message="<SILENCE>",
                                 templates=self.templates)
        
        return user_action
    
    # when the bot takes the role of API then, the User should assume the role of API_RESP (i.e API_RESPONSE)
    def api_response(self,bot_action) :
    
        user_action = None
        
        # if the API action asks for a account check
        if bot_action.get_description() == "API_AMOUNT_CHECK" :
            
            if self.user["amount"] > self.user["max_transferable_amt"] :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=["limit","balance","max_transferable_amt"],
                                     values={"limit" : self.user["limit"],
                                             "balance" : self.user["balance"],
                                             "max_transferable_amt" : self.user["max_transferable_amt"]},
                                     message="limit:{},balance:{},max_transferable_amt:{}, message:'change to max_transferable_amt ?'".format(self.user["limit"],
                                                                                                                                   self.user["balance"],
                                                                                                                                   self.user["max_transferable_amt"]),
                                     templates=self.templates)
            
            else :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=["limit","balance","max_transferable_amt"],
                                     values={"limit" : self.user["limit"],
                                             "balance" : self.user["balance"],
                                             "max_transferable_amt" : self.user["max_transferable_amt"]},
                                     message="api_result:success",
                                     templates=self.templates)
        
        # if the API action askes for a initial state check
        elif bot_action.get_description() == "API_INITIAL_SLOT_CHECK" :
            
            # if the flag becomes true at the end of this segment then it means that one or more than one slots are incorrect
            flag = False
            error_message = list()
            
            # if user account is given in the initial slots then check if it is appropriate
            if "user_account" in bot_action.get_slots() and self.user["user_account"] not in self.user["user_accounts"] :
                
                self.priority_states.append("check_account")
                self.priority_actions["check_account"] =  Action(actor="Bot",
                                                                action="api_call",
                                                                slots=["user_account"],
                                                                values=None,
                                                                message="api_call:account_check_api, user_account:{}".format(self.user["user_account"]),
                                                                description="API_ACCOUNT_CHECK",
                                                                templates=self.templates)
            
            # if destination name is given in the initial slots then check if it is appropriate
            if "destination_name" in bot_action.get_slots() and self.user["destination_name"] not in self.user["destination_names"] :
                
                self.priority_states.append("check_destination")
                self.priority_actions["check_destination"] = Action(actor="Bot",
                                                                   action="api_call",
                                                                   slots=["destination_name"],
                                                                   values=None,
                                                                   message="api_call:destination_name_check_api, destination_name:{}".format(self.user["destination_name"]),
                                                                   description="API_DESTINATION_NAME_CHECK",
                                                                   templates=self.templates)
            
            # if both user_account and amount are present then check if the amount satisfies the criteria
            if "user_account" in bot_action.get_slots() and "amount" in bot_action.get_slots() and self.user["amount"] > self.user["max_transferable_amt"] :
                
                self.priority_states.append("check_amount")
                self.priority_actions["check_amount"] = Action(actor="Bot",
                                              action="api_call",
                                              slots=["limit","balance"],
                                              values=None,
                                              message="api_call:amount_check_api, user_account:{}, amount:{}".format(self.user["user_account"],
                                                                      self.user["amount"]),
                                              description="API_AMOUNT_CHECK",
                                              templates=self.templates)
            
            # if self.priority_states is no empty then one or more than one value is incorrect then send appropriate error message
            if self.priority_states :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=self.priority_states,
                                     values=self.priority_actions,
                                     message="api_result:failed, message:'one or more slots are faulty'",
                                     templates=self.templates)
            
            # if everything is okay then send the correct message
            else :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=bot_action.get_slots(),
                                     values=None,
                                     message="api_result:success",
                                     templates=self.templates)
        
        # if the requested action is an account check
        elif bot_action.get_description() == "API_ACCOUNT_CHECK" :
            print("checking account")
            if self.user["user_account"] in self.user["user_accounts"] :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=["account"],
                                     values=self.user,
                                     message="api_result:success",
                                     templates=self.templates)
            
            else :
                
                slot_message = ','.join(self.user["user_accounts"])
                api_message = "api_result:failed, message:'availbale list of user accounts : {}'".format(slot_message)
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=self.user["user_accounts"],
                                     values=self.user,
                                     message=api_message,
                                     templates=self.templates)
        
        # if the requested action is destination name check
        elif bot_action.get_description() == "API_DESTINATION_NAME_CHECK" :
            
            if self.user["destination_name"] in self.user["destination_names"] :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=["destination_name"],
                                     values=None,
                                     message="api_result:success",
                                     templates=self.templates)
            
            else :
                
                slot_message = ','.join(self.user["destination_names"])
                api_message = "api_result:failed, message:'available list of names :{}'".format(slot_message)
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=self.user["destination_names"],
                                     values={"destination_names" : self.user["destination_names"]},
                                     message=api_message,
                                     templates=self.templates)
        else :
            user_action = self.perform_random_action(bot_action)
        
        return user_action            

In [7]:
class Transaction_bot(object) :
    
    def __init__(self,templates=None) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","destination_name","amount"]
        self.slots_to_ask = ["user_account","destination_name","amount"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "check_initial" : self.check_initial_state,
                       "user_account" : self.account_state ,
                       "check_account" : self.check_account_state,
                       "change_account" : self.change_account_state,
                       "destination_name" : self.destination_state ,
                       "check_destination" : self.check_destination_name_state,
                       "change_destination_name" : self.change_destination_name_state,
                       "amount" : self.amount_state ,
                       "check_amount" : self.check_amount_state,
                       "change_amount" : self.change_amount_state,
                       "end_call" : self.end_call_state ,
                       "balance" : self.balance_state ,
                       "confirmation_state" : self.confirmation_state,
                       "api_call" : self.api_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        self.templates = templates
        self.current_state = self.initial_state
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "destination_name" in slots_given :
            slots_sorted.append("destination_name")
            slots_given.remove("destination_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        for slot in user_action.get_slots() :
            
            if slot in self.slots_to_ask :
                
                self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        #print("next state is  == > {}".format(next_state))
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    def initial_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_slots() :
            
            if "intent" in user_action.get_slots() and len(user_action.get_slots()) > 1 :
                
                next_state = "check_initial"
                slots_given = user_action.get_slots()[1:] 
                slot_message = "api_call:initial_slot_check,"
                for slot in slots_given :
                    slot_message += " {}:{},".format(slot,self.user_values[slot])
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=user_action.get_slots(),
                                    values=None,
                                    message=slot_message[:-1],
                                    description="API_INITIAL_SLOT_CHECK",
                                    templates=self.templates)
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting the values for {}".format(next_state),
                                    templates=self.templates)        
        else :           
            
            next_state = "initial"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["intent"],
                                values=None,
                                message="Get the intent first",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def check_initial_state(self,user_action) :
        # if the below message is received then it means that initial check is successful and move on to the next appropriate slots
        
        if user_action.get_message() == "api_result:success" :
            
            if not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?",
                                    templates=self.templates)
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    templates=self.templates)
        
        else :
            
            self.priority_states = user_action.get_slots()
            self.priority_actions = user_action.get_values()
            
            next_state = self.priority_states[0]
            bot_action = self.priority_actions[next_state]
            
            self.priority_states.remove(next_state)
        
        return next_state , bot_action
    
    def account_state(self,user_action) :
        
        # if user account has been given then 
        if "user_account" in user_action.get_slots() :
            
            # remove the slot which has already been asked
            self.remove_informed_slots(user_action)
                
            # update user infomation
            user_values = user_action.get_values()
            
            # record and store all the values given by the user
            self.record_user_values(user_action)
            
            
            # pick a slot to ask randomly from the remaining slots_to_ask
            next_state = "check_account"
            
            # perform the corresponding bot information
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["user_account"],
                                values=None,
                                message="api_call:account_check_api, user_account:{}".format(self.user_values["user_account"]),
                                description="API_ACCOUNT_CHECK",
                                templates=self.templates)
                    
        else :
            
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["user_account"],
                                values=None,
                                message="requesting user to specify the account",
                                templates=self.templates)
            
        return next_state , bot_action
    
    def check_account_state(self,user_action) :
        
        if user_action.get_message() == "api_result:success" :
            
            if "amount" in self.user_values.keys() :
                
                next_state = "check_amount"
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["limit","balance"],
                                    values=None,
                                    message="api_call:amount_check_api, user_account:{}, amount:{}".format(self.user_values["user_account"],self.user_values["amount"]),
                                    description="API_AMOUNT_CHECK",
                                    templates=self.templates)
            
            else :
                
                if self.priority_states :
                    next_state = self.priority_states[0]
                    bot_action = self.priority_actions[next_state]
                
                    self.priority_states.remove(next_state)
            
                elif not self.slots_to_ask :
                    
                    next_state = "confirmation_state"
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=None,
                                        values=None,
                                        message="confirm transaction ?",
                                        templates=self.templates)
            
                else :
                    
                    next_state = self.slots_to_ask[0]
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=[next_state],
                                        values=None,
                                        message="request for {}".format(next_state),
                                        templates=self.templates)        
        else :
            
            next_state = "change_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "It seems that you have not entered a valid account, you available accounts are {}, would you like change the source account ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_ACCOUNT",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def change_account_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=[next_state],
                                values=None,
                                message="Requesting user to provide new user account",
                                templates=self.templates)
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="You denied to change the account",
                                templates=self.templates)
        
        return next_state , bot_action 
    
    def destination_state(self,user_action) :
        # remove the slot already asked
        self.remove_informed_slots(user_action)
            
        # update the user information with the new values got
        self.record_user_values(user_action)
        
        if "destination_name" in user_action.get_slots() :
            
            # sample out a new state based on the remaining slots to ask
            next_state = "check_destination"
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["destination_name"],
                                values=None,
                                message="api_call:destination_check_api, destination_name:{}".format(self.user_values["destination_name"]),
                                description="API_DESTINATION_NAME_CHECK",
                                templates=self.templates)
        
        else :
            
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["destination_name"],
                                values=None,
                                message="provide the Name of the Receiver",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def check_destination_name_state(self,user_action) :
        
        if user_action.get_message() == "api_result:success" :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
                
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?",
                                    templates=self.templates)
                
            else :
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="Requesting the name for {}".format(next_state),
                                    templates=self.templates)
        
        else :
            
            next_state = "change_destination_name"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "The recipient you are trying to provide doesn't exist, available list of recipients is {}, would you like to change the recipient ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_DESTINATION_NAME",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def change_destination_name_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=[next_state],
                                values=None,
                                message="Requesting user to provide new user account",
                                templates=self.templates)
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="User failed to change the account",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "amount" in user_action.get_slots() :
             
            if "user_account" in self.user_values.keys() :
                
                # No random check this time because we have to check if the amount given is correct or not
                next_state = "check_amount"
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=["limit","balance"],
                                    values=None,
                                    message="api_call:check_amount, user_account:{}, amount:{}".format(self.user_values["user_account"],self.user_values["amount"]),
                                    description="API_AMOUNT_CHECK",
                                    templates=self.templates)
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting user to provide {} ".format(next_state),
                                    templates=self.templates)
        
        else :
            
            next_state = "amount"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["amount"],
                                values=None,
                                message="requesting the user to provide the Amount",
                                templates=self.templates)
            
        return next_state , bot_action

    
    def check_amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        
        if user_action.get_message() == "api_result:success" :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?",
                                    templates=self.templates)
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    templates=self.templates)

        else :
            
            next_state = "change_amount"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message="It seems the amount you provided can't be processed because your transaction limit is {} and your current balance is {} so the maximum you can transfer is {}, would you like to reduce your amount to this amount ?".format(self.user_values["limit"],self.user_values["balance"],self.user_values["max_transferable_amt"]),
                                description="CHANGE_TO_MAX_TRANSFERABLE_AMT",
                                templates=self.templates)
        
        return next_state , bot_action
    
    
    def change_amount_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            self.user_values["amount"] = self.user_values["max_transferable_amt"]
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?",
                                    templates=self.templates)
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state),
                                    templates=self.templates)
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="Rejected to change the decision",
                                templates=self.templates)
        
        return next_state , bot_action
    
    # This is a dead state and I don't know why it is here , but I don't know what will happen if I remove this function
    def balance_state(self,user_action) :
        return
    
    # ask for confirmation
    def confirmation_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "api_call"
            #api_value = self.user_values["user_account"] + " " + self.user_values["destination_name"] + " "  + str(self.user_values["amount"])
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=["user_account","destination_name","amount"],
                                values={"user_account" : self.user_values["user_account"],
                                        "destination_name" : self.user_values["destination_name"],
                                        "amount" : self.user_values["amount"]},
                                message="api_call:transaction_api, user_account:{}, destination_name:{}, amount:{}".format(self.user_values["user_account"],self.user_values["destination_name"],self.user_values["amount"]),
                                description="API_CALL",
                                templates=self.templates)
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="User refused to confirm transaction",
                                templates=self.templates)
        
        return next_state, bot_action

    # end the call
    def end_call_state(self,user_action) :
        
        print("inside end_call state")
        next_state = "initial"
        bot_action = None
        
        return next_state , bot_action
    
    # Api call state
    def api_call_state(self,user_action) :
        
        if user_action.get_message() == "api_result:success" :
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="{} conducted successfully, ciao !!".format(self.user_values["intent"]),
                                templates=self.templates)
        
        else :
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="error in processing {}".format(self.user_values["intent"]),
                                templates=self.templates)
        
        next_state = "end_call"
        
        return next_state , bot_action

In [8]:
def create_dialogs(User,Bot,number_of_dialogs,dialog_templates=None) :
    
    dialogs = list()
    
    for i in range(number_of_dialogs) :
        
        dialog = list()

        user = User(templates=dialog_templates)
        bot = Bot(templates=dialog_templates)

        user_action = Action(actor="User",
                             action=None,
                             slots=None,
                             values=None,
                             message="<SILENCE>",
                             templates=dialog_templates)
        

        bot_action = Action(actor="Bot",
                            action="request",
                            slots=["intent"],
                            values=None,
                            message="Gettinng intent",
                            templates=dialog_templates)
        
        
        
        dialog.append(user_action)
        dialog.append(bot_action)
        
        latest_action = None
        
        while latest_action != "end_call" :
            user_action = user.speak(bot_action)
            print("user_dialog: {}".format(user_action.get_dialog()))
            #print("user_action {}, user_message {} user_description {}".format(user_action.get_action(),user_action.get_message(),user_action.get_description()))
            bot_action = bot.speak(user_action)
            print("bot_dialog: {}".format(bot_action.get_dialog()))
            #print("bot_action {}, bot_message {} bot_description {}".format(bot_action.get_action(),bot_action.get_message(),bot_action.get_description()))
            latest_action = bot_action.get_action() 
            #print("latest action is {}".format(latest_action))
            dialog.append(user_action)
            dialog.append(bot_action)
            #print("User:{} Bot:{}".format(user_action.get_message(),bot_action.get_message()))
        
        dialogs.append(dialog)
    
    return dialogs

In [9]:
transaction_dialogs = create_dialogs(User=Transaction_user,Bot=Transaction_bot,number_of_dialogs=100,dialog_templates=transaction_templates)

user_dialog: User : I need to transfer money
bot_dialog: Bot : From which account ?
user_dialog: User : use the Credit for the transaction
bot_dialog: Bot : api_call:account_check_api, user_account:Credit
checking account
user_dialog: API : api_response api_result:failed, message:'availbale list of user accounts : Savings'
bot_dialog: Bot : It seems that you have not entered a valid account, you available accounts are Savings, would you like change the source account ?
user_dialog: User : reject
bot_dialog: Bot : end_call You denied to change the account
user_dialog: User : Transfer 2400
bot_dialog: Bot : api_call:initial_slot_check, amount:2400
user_dialog: API : api_response api_result:success
bot_dialog: Bot : From which account ?
user_dialog: User : withdraw from Checkin
bot_dialog: Bot : api_call:account_check_api, user_account:Checkin
checking account
user_dialog: API : api_response api_result:failed, message:'availbale list of user accounts : Credit'
bot_dialog: Bot : It seems t

In [10]:
def create_raw_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
    
    file_handle = open(os.path.join(file_directory,file_name),"w")
    
    for dialog in dialogs :
        for action in dialog :
            file_handle.write(action.get_dialog())
            file_handle.write("\n")
        file_handle.write("\n")
    file_handle.close()

In [11]:
create_raw_data(file_directory="../data/transaction_data/",file_name="raw_data.txt",dialogs=transaction_dialogs)

In [12]:
def create_training_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for dialog in dialogs :
        count = 1
        for i in range(0,len(dialog),2) :
            file_handle.write("{} {}\t{}\n".format(str(count),dialog[i].get_dialog(with_actor=False),dialog[i+1].get_dialog(with_actor=False)))
            count += 1
        file_handle.write("\n")
    file_handle.close()

In [13]:
create_training_data(file_directory="../data/transaction_data/",file_name="train_data.txt",dialogs=transaction_dialogs)
create_training_data(file_directory="../data/transaction_data/",file_name="val_data.txt",dialogs=transaction_dialogs)
create_training_data(file_directory="../data/transaction_data/",file_name="test_data.txt",dialogs=transaction_dialogs)

In [14]:
def create_candidates(file_directory="../data/",file_name="data.txt",dialogs=None) :
    candidate_set = set()
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    for dialog in dialogs :
        for action in dialog :
            if action.get_actor() == "Bot" :
                candidate_set.add(action.get_dialog(with_actor=False))
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for candidate in candidate_set :
        file_handle.write("1 {}\n".format(candidate))
    file_handle.close()

In [15]:
create_candidates(file_directory="../data/transaction_data/",file_name="candidates.txt",dialogs=transaction_dialogs)